In [87]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
import networkx as nx
import shapely
import folium
import geojson

from shapely.ops import unary_union
from shapely.geometry import Polygon, MultiPolygon, LineString, Point
from geopy.distance import geodesic

In [3]:
# Defining classes for the dataframes
class AmenityPoint:
    def __init__(self, geometry, lat, lon, amenity, name, addr_city):
        self.geometry = geometry
        self.lat = lat
        self.lon = lon
        self.amenity = amenity
        self.name = name
        self.addr_city = addr_city

class AmenityPolygon:
    def __init__(self, geometry, lat, lon, amenity, name, addr_city):
        self.geometry = geometry
        self.lat = lat
        self.lon = lon
        self.amenity = amenity
        self.name = name
        self.addr_city = addr_city

In [5]:
# Read the building footprints data

buildingfootprints_gdf = gpd.read_file('manila_building_footprints.geojson')

buildingfootprints_gdf.head()

,id,geometry
0,0,"POLYGON ((120.95954 14.62190, 120.95945 14.621..."
1,1,"POLYGON ((120.96084 14.62342, 120.96095 14.623..."
2,2,"POLYGON ((120.96181 14.60838, 120.96172 14.608..."
3,3,"POLYGON ((120.96087 14.62855, 120.96103 14.628..."
4,4,"POLYGON ((120.96158 14.62105, 120.96162 14.621..."


In [6]:
# Load the Manila amenities data into a Geopandas dataframe
from shapely import wkt

manila_amenities_df = pd.read_csv('manila_amenities.csv')
manila_amenities_df['geometry'] = manila_amenities_df['geometry'].apply(wkt.loads)
manila_amenities_gdf = gpd.GeoDataFrame(manila_amenities_df, crs='epsg:4326')

# Separate into point and polygon dataframes
manila_amenities_polygon_gdf = manila_amenities_gdf[manila_amenities_gdf['geometry'].geom_type == 'Polygon']
manila_amenities_point_gdf = manila_amenities_gdf[manila_amenities_gdf['geometry'].geom_type == 'Point']
manila_amenities_multipoly_gdf = manila_amenities_gdf[manila_amenities_gdf['geometry'].geom_type == 'MultiPolygon']

# Append multipolygons to the polygon dataframe
manila_amenities_polygon_gdf = gpd.GeoDataFrame(pd.concat([manila_amenities_polygon_gdf, manila_amenities_multipoly_gdf], ignore_index=True))

# Reset point dataframe index
manila_amenities_point_gdf.reset_index(drop=True, inplace=True)

# Add a column to the polygon dataframe to store a list of Amenity Points within the polygon
manila_amenities_polygon_gdf['amenity_points'] = None

In [7]:
# For each polygon in the polygon dataframe, find all the points from the point dataframe lying inside that polygon
# Store the list of points in the 'amenity_points' column of the polygon dataframe as a list of point indices
for i, polygon in manila_amenities_polygon_gdf.iterrows():
    points_within_polygon = []

    for j, point in manila_amenities_point_gdf.iterrows():
        try:
            if polygon['geometry'].intersects(point['geometry']):
                # Append the index of the current point
                points_within_polygon.append(j)
        except Exception as e:
            print(f"Error processing polygon {i} point {j}: {e}")
    manila_amenities_polygon_gdf.at[i, 'amenity_points'] = points_within_polygon

In [8]:
manila_amenities_polygon_gdf['amenity'].unique()

array(['education', 'finance', 'government offices', 'grocery', 'health',
       'malls', 'residential areas', 'security'], dtype=object)

In [9]:
manila_amenities_polygon_gdf[manila_amenities_polygon_gdf['name'].isnull()]

,amenity,addr_city,name,x,y,geometry,amenity_points
1,education,Manila,NaN,120.988929,14.603745,"POLYGON ((120.98881 14.60384, 120.98885 14.603...",[81]
144,education,Manila,NaN,120.992154,14.560229,"POLYGON ((120.99219 14.56033, 120.99224 14.560...",[]
196,education,Manila,NaN,120.979357,14.610687,"POLYGON ((120.97977 14.61028, 120.97945 14.610...",[]
219,education,Manila,NaN,121.008584,14.580380,"POLYGON ((121.00851 14.58041, 121.00863 14.580...",[]
223,education,Manila,NaN,121.008665,14.572657,"POLYGON ((121.00871 14.57288, 121.00878 14.572...",[]
...,...,...,...,...,...,...,...
1227,residential areas,Manila,NaN,120.977209,14.608424,"POLYGON ((120.97721 14.60855, 120.97729 14.608...",[]
1239,security,Manila,NaN,120.982806,14.607571,"POLYGON ((120.98266 14.60765, 120.98296 14.607...",[]
1265,security,Manila,NaN,120.970972,14.602583,"POLYGON ((120.97090 14.60256, 120.97100 14.602...",[]
1266,security,Manila,NaN,120.959548,14.630220,"POLYGON ((120.95940 14.63039, 120.95971 14.630...",[]


In [135]:
# Reading population data
manila_population_df = pd.read_csv('manila-population-polygon.csv')
manila_population_df['geometry'] = manila_population_df['geometry'].apply(wkt.loads)
manila_population_gdf = gpd.GeoDataFrame(manila_population_df, crs='epsg:4326')

# Create a base map centered around Manila
map_center = (14.599512, 120.984222) # TEMPORARY WILL ZOOM TO MANILA
m = folium.Map(location=map_center, zoom_start=10, tiles='openstreetmap')

# Add points to the map
for index, row in manila_population_gdf.iterrows():
    folium.CircleMarker(location=[row['latitude'], row['longitude']],
                        radius=1,  # Adjust the radius as needed for population density representation
                        color='blue',  # Change color as needed
                        fill=True,
                        fill_color='blue'  # Change fill color as needed
                        ).add_to(m)
    
m.save('population.html') # Save the map to an HTML file

In [62]:
# Buckle up. We're trying to create a network out of this monstrosity of a dataframe

# Create a networkx graph

def create_network(amenities_polygon_gdf, amenities_point_gdf):
    amenities_network = nx.Graph()

   # Add polygon nodes
    for index, row in amenities_polygon_gdf.iterrows():
        # Check if essential columns exist in the row
        if 'geometry' in row and 'amenity' in row and 'name' in row and 'addr_city' in row and 'amenity_points' in row:
            # Generate a unique node identifier for polygons
            node_id = f"polygon_{index}"
            amenities_network.add_node(node_id, polygon_index=index, geometry=row['geometry'], lat=row['geometry'].centroid.y, lon=row['geometry'].centroid.x, amenity=row['amenity'], name=row.get('name', ''), addr_city=row['addr_city'], amenity_points=row['amenity_points'])
        else:
            print(f"Skipping row {index} in amenities_polygon_gdf due to missing data.")

    # Add point nodes
    for index, row in amenities_point_gdf.iterrows():
        # Check if essential columns exist in the row
        if 'geometry' in row and 'amenity' in row and 'name' in row and 'addr_city' in row:
            # Generate a unique node identifier for points
            node_id = f"point_{index}"
            amenities_network.add_node(node_id, point_index=index, geometry=row['geometry'], lat=row['y'], lon=row['x'], amenity=row['amenity'], name=row.get('name', ''), addr_city=row['addr_city'], is_in_polygon=False)
        else:
            print(f"Skipping row {index} in amenities_point_gdf due to missing data.")
            
    return amenities_network

In [19]:
# #ORIGINAL
# # NOTABLE HYPERPARAMETERS:
# # Amenities are connected based on a radius from the centroid of the polygon
# # Aggregated polygon length (perimeter) is used to check if zone is sufficiently large  

# def combine_contiguous_amenities_by_polygon2(graph, max_distance, max_perimeter):
#     for node in list(graph.nodes(data=True)):
#         if node[1]['geometry'].geom_type == 'Polygon':
#             other_node_distances = []
#             contained_points = []
#             for other_node in list(graph.nodes(data=True)):
#                 if other_node[1]['geometry'].geom_type in ['Polygon', 'MultiPolygon']:
#                     if node[0] != other_node[0] and node[1]['amenity'] == other_node[1]['amenity']:
#                         if node[1]['geometry'].centroid.distance(other_node[1]['geometry']) < max_distance:
#                             other_node_distances.append((other_node[0], node[1]['geometry'].centroid.distance(other_node[1]['geometry'])))
#                 elif other_node[1]['geometry'].geom_type == 'Point':
#                     if node[1]['geometry'].intersects(other_node[1]['geometry']) and not other_node[1]['is_in_polygon']:
#                         contained_points.append(other_node[0])
#             other_node_distances.sort(key=lambda x: x[1])

#             if other_node_distances:
#                 for new_node, new_distance in other_node_distances:
#                     combined_node = shapely.ops.unary_union([node[1]['geometry'], graph.nodes[new_node]['geometry']])
#                     if combined_node.length < max_perimeter:
#                         graph.nodes[node[0]]['geometry'] = combined_node
#                         if isinstance(graph.nodes[node[0]]['name'], float): # Temporary for null values
#                             graph.nodes[node[0]]['name'] = 'null, ' + graph.nodes[new_node]['name']
#                         else:
#                             graph.nodes[node[0]]['name'] = graph.nodes[node[0]]['name'] + ', ' + graph.nodes[new_node]['name']
#                         graph.nodes[node[0]]['lat'] = combined_node.centroid.y
#                         graph.nodes[node[0]]['lon'] = combined_node.centroid.x
#                         graph.nodes[node[0]]['amenity_points'] += graph.nodes[new_node]['amenity_points']
#                         graph.remove_node(new_node)
#                         break

#             for point in contained_points:
#                 graph.nodes[point]['is_in_polygon'] = True

#     return graph

In [142]:
from shapely.ops import unary_union
from shapely.geometry import Polygon, MultiPolygon, LineString
from geopy.distance import geodesic

def combine_contiguous_amenities_by_polygon(graph, max_distance, max_perimeter):
    for node_key, node_data in list(graph.nodes.items()):
        if 'geometry' in node_data and node_data['geometry'].geom_type in ['Polygon', 'MultiPolygon']:
            other_node_distances = []
            contained_points = []
            for other_node_key, other_node_data in list(graph.nodes.items()):
                if 'geometry' in other_node_data and other_node_data['geometry'].geom_type in ['Polygon', 'MultiPolygon']:
                    if node_key != other_node_key and node_data['amenity'] == other_node_data['amenity']:
                        
                        # Distance between the two centroids in meters
                        c1 = (node_data['geometry'].centroid.y, node_data['geometry'].centroid.x)
                        c2 = (other_node_data['geometry'].centroid.y, other_node_data['geometry'].centroid.x)
                        distance = geodesic(c1, c2).meters

                        if distance < max_distance:

                            # Check if any other amenities intersect the line between centroids
                            line_between_centroids = LineString([node_data['geometry'].centroid, other_node_data['geometry'].centroid])
                            amenities_intersecting = any(graph.nodes[amenity_key]['geometry'].intersects(line_between_centroids) for amenity_key in graph.nodes if amenity_key != node_key and amenity_key != other_node_key)
                            if not amenities_intersecting:
                                other_node_distances.append((other_node_key, node_data['geometry'].centroid.distance(other_node_data['geometry'])))
                                
                                # Checks if contiguous, if not add a line
                                if not graph.nodes[node_key]['geometry'].intersects(graph.nodes[other_node_key]['geometry']):
                                    connect_polygon_lines(node_data['geometry'], other_node_data['geometry'])
                
                elif 'geometry' in other_node_data and other_node_data['geometry'].geom_type == 'Point':
                    if node_data['geometry'].intersects(other_node_data['geometry']) and not other_node_data.get('is_in_polygon', False):
                        contained_points.append(other_node_key)
            other_node_distances.sort(key=lambda x: x[1])

            if other_node_distances:
                for new_node_key, new_distance in other_node_distances:
                    if new_node_key in graph.nodes and node_key in graph.nodes:  # Check if new_node_key exists in the graph
                        combined_node = shapely.ops.unary_union([node_data['geometry'], graph.nodes[new_node_key]['geometry']])
                        if combined_node.length < max_perimeter:
                            graph.nodes[node_key]['geometry'] = combined_node
                            graph.nodes[node_key]['name'] = combine_names(graph.nodes[node_key].get('name'), graph.nodes[new_node_key].get('name'))
                            graph.nodes[node_key]['lat'] = combined_node.centroid.y
                            graph.nodes[node_key]['lon'] = combined_node.centroid.x
                            graph.nodes[node_key]['amenity_points'] += graph.nodes[new_node_key].get('amenity_points', 0)
                            graph.remove_node(new_node_key)

            for point_key in contained_points:
                graph.nodes[point_key]['is_in_polygon'] = True

# TEMPORARY SOLUTION FOR NULL NAMES
def combine_names(name1, name2):
    # Combine names ensuring that no null values are included
    if isinstance(name1, str) and isinstance(name2, str):
        return f"{name1}, {name2}"
    elif isinstance(name1, str):
        return name1
    elif isinstance(name2, str):
        return name2
    else:
        return None
    
#FUNCTION FOR VISUALIZATION
def connect_polygon_lines(p1, p2):
    centroid1 = p1.centroid
    centroid2 = p2.centroid

    # Create a LineString from the centroids
    line_between_polygons = LineString([centroid1, centroid2])
    connected_lines.append(line_between_polygons)

In [18]:
# # MERGE NON-CONTIGUOUS SHAPES VERSION
# #Improved code -> Creates a copy of a graph and Merges non-contiguous and non-overlapping shapes

# from shapely.ops import unary_union
# from shapely.geometry import Polygon, MultiPolygon, LineString
# from geopy.distance import geodesic

# def merge_amenities_by_polygon(graph, max_distance, max_perimeter):
#     combined_graph = nx.Graph()
#     list_to_merge = []
    
#     for node_key, node_data in list(graph.nodes.items()):
#         if 'geometry' in node_data and node_data['geometry'].geom_type == 'Polygon':
#             nodes_to_merge = []
#             contained_points = []
            
#             #Check first if it is already in a list of polygons to be merged
#             for merge_list in list_to_merge:
#                 if node_key in merge_list:
#                     nodes_to_merge = merge_list
#                     break
            
#             # If this is a new node that is not part of any list, add itself to the list for merging later
#             if not nodes_to_merge:
#                 nodes_to_merge.append(node_key)
            
#             for other_node_key, other_node_data in list(graph.nodes.items()):
#                 if 'geometry' in other_node_data and other_node_data['geometry'].geom_type in ['Polygon', 'MultiPolygon']:
#                     # Check if its not the same node, it is the same amenity, and is not already in the list to merge
#                     if node_key != other_node_key and node_data['amenity'] == other_node_data['amenity'] and other_node_key not in nodes_to_merge:
                        
#                         # Distance between the two centroids in meters
#                         c1 = (node_data['geometry'].centroid.y, node_data['geometry'].centroid.x)
#                         c2 = (other_node_data['geometry'].centroid.y, other_node_data['geometry'].centroid.x)
#                         distance = float(geodesic(c1, c2).meters)

#                         if distance < max_distance:

#                             # Check if any other amenities intersect the line between centroids
#                             line_between_centroids = LineString([node_data['geometry'].centroid, other_node_data['geometry'].centroid])
#                             amenities_intersecting = any(graph.nodes[amenity_key]['geometry'].intersects(line_between_centroids) for amenity_key in graph.nodes if amenity_key != node_key and amenity_key != other_node_key)
#                             if not amenities_intersecting:
#                                 nodes_to_merge.append(other_node_key)
                
#                 elif 'geometry' in other_node_data and other_node_data['geometry'].geom_type == 'Point':
#                     #Add the point to the new graph if it is not there yet
#                     if other_node_key not in combined_graph.nodes:
#                         combined_graph.add_node(other_node_key, **other_node_data)
                        
#                     if node_data['geometry'].intersects(other_node_data['geometry']) and not other_node_data.get('is_in_polygon', False):
#                         contained_points.append(other_node_key)
            
#             if nodes_to_merge not in list_to_merge:
#                 list_to_merge.append(nodes_to_merge) # Add to the list to merge the polygons later

#             for point_key in contained_points:
#                 graph.nodes[point_key]['is_in_polygon'] = True
                
                
                
#     # Now we will finally merge all polygons in the list
#     for merge_list in list_to_merge:
#         first = True
#         for node_key in merge_list:
#             if first:
#                 combined_node = graph.nodes[node_key]['geometry']
#                 combined_node_amenity = graph.nodes[node_key]['amenity']
#                 combined_node_key = node_key
#                 combined_node_geometry = graph.nodes[node_key]['geometry']
#                 combined_node_name = graph.nodes[node_key]['name']
#                 combined_node_lat = graph.nodes[node_key]['lat']
#                 combined_node_lon = graph.nodes[node_key]['lon']
#                 combined_node_points = graph.nodes[node_key]['amenity_points']
#                 first = False
#             else:
#                 combined_node = merge_polygons([combined_node, graph.nodes[node_key]['geometry']])
#                 if combined_node.length < max_perimeter:
#                     combined_node_geometry = merge_polygons([combined_node_geometry, graph.nodes[node_key]['geometry']])
#                     combined_node_name = combine_names(combined_node_name, graph.nodes[node_key].get('name'))
#                     combined_node_lat = combined_node_geometry.centroid.x
#                     combined_node_lon = combined_node_geometry.centroid.x
#                     combined_node_points += graph.nodes[node_key].get('amenity_points', 0)
                
#         combined_graph.add_node(combined_node_key, geometry=combined_node_geometry, name=combined_node_name, lat=combined_node_lat, amenity=combined_node_amenity,
#                                 lon=combined_node_lon, amenity_points=combined_node_points)

#     return combined_graph

# # TEMPORARY SOLUTION FOR NULL NAMES
# def combine_names(name1, name2):
#     # Combine names ensuring that no null values are included
#     if isinstance(name1, str) and isinstance(name2, str):
#         return f"{name1}, {name2}"
#     elif isinstance(name1, str):
#         return name1
#     elif isinstance(name2, str):
#         return name2
#     else:
#         return None


# # MERGING NON-CONTIGUOUS AND NON-OVERLAPPING POLYGONS
# def merge_polygons(geometries):
#     if not geometries:
#         return None
    
#     return MultiPolygon(geometries).convex_hull 
        

In [ ]:
# # Merge overlapping polygons -> To be used with the merging of polygons
# def merge_overlapping_polygons(graph):
#     for node_key, node_data in list(graph.nodes.items()):
#         if 'geometry' in node_data and node_data['geometry'].geom_type == 'Polygon':
#             for other_node_key, other_node_data in list(graph.nodes.items()):
#                   if 'geometry' in other_node_data and other_node_data['geometry'].geom_type in ['Polygon', 'MultiPolygon']:
#                     # Check if its not the same node, it is the same amenity, and is not already in the list to merge
#                     if node_key != other_node_key and node_data['amenity'] == other_node_data['amenity']:

#                         # Convert to polygons
#                         p1 = Polygon(node_data['geometry'])
#                         p2 = Polygon(other_node_data['geometry'])
                        
#                         if p1.intersects(p2):
                            
                

In [74]:
# # Lines version - Non-Merging
# # Creates a copy of a graph and connects non-contiguous and non-overlapping shapes instead of merging

# def combine_amenities_by_polygon(graph, max_distance, max_perimeter):
#     combined_graph = nx.Graph()
#     list_to_merge = []
    
#     for node_key, node_data in list(graph.nodes.items()):
#         if 'geometry' in node_data and node_data['geometry'].geom_type in ['Polygon', 'MultiPolygon']:
#             nodes_to_merge = []
#             contained_points = []
            
#             #Check first if it is already in a list of polygons to be merged
#             for merge_list in list_to_merge:
#                 if node_key in merge_list:
#                     nodes_to_merge = merge_list
#                     break
            
#             # If this is a new node that is not part of any list, add itself to the list for merging later
#             if not nodes_to_merge:
#                 nodes_to_merge.append(node_key)
            
#             for other_node_key, other_node_data in list(graph.nodes.items()):
#                 if 'geometry' in other_node_data and other_node_data['geometry'].geom_type in ['Polygon', 'MultiPolygon']:
#                     # Check if its not the same node, it is the same amenity, and is not already in the list to merge
#                     if node_key != other_node_key and node_data['amenity'] == other_node_data['amenity'] and other_node_key not in nodes_to_merge:
                        
#                         # Distance between the two centroids in meters
#                         c1 = (node_data['geometry'].centroid.y, node_data['geometry'].centroid.x)
#                         c2 = (other_node_data['geometry'].centroid.y, other_node_data['geometry'].centroid.x)
#                         distance = float(geodesic(c1, c2).meters)

#                         if distance < max_distance:

#                             # Check if any other amenities intersect the line between centroids
#                             line_between_centroids = LineString([node_data['geometry'].centroid, other_node_data['geometry'].centroid])
#                             amenities_intersecting = any(graph.nodes[amenity_key]['geometry'].intersects(line_between_centroids) for amenity_key in graph.nodes if amenity_key != node_key and amenity_key != other_node_key)
#                             if not amenities_intersecting:
#                                 nodes_to_merge.append(other_node_key)
                                
#                                 # Checks if contiguous, if not add a line
#                                 if not graph.nodes[node_key]['geometry'].intersects(graph.nodes[other_node_key]['geometry']):
#                                     connect_polygon_lines(node_data['geometry'], other_node_data['geometry'])
                
#                 elif 'geometry' in other_node_data and other_node_data['geometry'].geom_type == 'Point':
#                     #Add the point to the new graph if it is not there yet
#                     if other_node_key not in combined_graph.nodes:
#                         combined_graph.add_node(other_node_key, **other_node_data)
                        
#                     if node_data['geometry'].intersects(other_node_data['geometry']) and not other_node_data.get('is_in_polygon', False):
#                         contained_points.append(other_node_key)
            
#             if nodes_to_merge not in list_to_merge:
#                 list_to_merge.append(nodes_to_merge) # Add to the list to merge the polygons later

#             for point_key in contained_points:
#                 graph.nodes[point_key]['is_in_polygon'] = True
                
                
                
#     # Now we will finally connect all polygons in the list
#     for merge_list in list_to_merge:
#         first = True
#         for node_key in merge_list:
#             print("Merging ", graph.nodes[node_key]['name'])
#             if first:
#                 combined_node = graph.nodes[node_key]['geometry']
#                 combined_node_amenity = graph.nodes[node_key]['amenity']
#                 combined_node_key = node_key
#                 combined_node_geometry = graph.nodes[node_key]['geometry']
#                 combined_node_name = graph.nodes[node_key]['name']
#                 combined_node_lat = graph.nodes[node_key]['lat']
#                 combined_node_lon = graph.nodes[node_key]['lon']
#                 combined_node_points = graph.nodes[node_key]['amenity_points']
#                 first = False
#             else:
#                 combined_node = shapely.ops.unary_union([combined_node_geometry, graph.nodes[node_key]['geometry']])
#                 if combined_node.length < max_perimeter:
#                     combined_node_geometry = combined_node
#                     combined_node_name = combine_names(combined_node_name, graph.nodes[node_key].get('name'))
#                     combined_node_lat = combined_node_geometry.centroid.x
#                     combined_node_lon = combined_node_geometry.centroid.x
#                     combined_node_points += graph.nodes[node_key].get('amenity_points', 0)
                
#         combined_graph.add_node(combined_node_key, geometry=combined_node_geometry, name=combined_node_name, lat=combined_node_lat, amenity=combined_node_amenity,
#                                 lon=combined_node_lon, amenity_points=combined_node_points)

#     return combined_graph

# # TEMPORARY SOLUTION FOR NULL NAMES
# def combine_names(name1, name2):
#     # Combine names ensuring that no null values are included
#     if isinstance(name1, str) and isinstance(name2, str):
#         return f"{name1}, {name2}"
#     elif isinstance(name1, str):
#         return name1
#     elif isinstance(name2, str):
#         return name2
#     else:
#         return None
        
# #FUNCTION FOR VISUALIZATION
# def connect_polygon_lines(p1, p2):
#     centroid1 = p1.centroid
#     centroid2 = p2.centroid

#     # Create a LineString from the centroids
#     line_between_polygons = LineString([centroid1, centroid2])
#     connected_lines.append(line_between_polygons)

In [20]:
def plot_network_on_map(amenities_network, initial_location=[0, 0], zoom_start=10):
    # Create a map centered at the initial location
    map_center = (14.599512, 120.984222) # TEMPORARY WILL ZOOM TO MANILA
    m = folium.Map(location=map_center, zoom_start=zoom_start, tiles='openstreetmap')
    
    #Colours for Visualization
    amenity_colors = {
        'education': 'green',
        'finance': 'blue',
        'government offices': 'red',
        'grocery': 'orange',
        'health': 'magenta',
        'malls': 'yellow',
        'residential areas': 'brown',
        'security': 'gray'
    }

    # Iterate over the nodes in the network
    for node, data in amenities_network.nodes(data=True):
        # Check if the node has a geometry attribute
        if 'geometry' in data:
            # Get the geometry of the node
            geometry = data['geometry']

            # Check the geometry type and plot accordingly
            if geometry.geom_type == 'Point':
                # Plot a marker for points    
                #folium.Marker(location=[geometry.y, geometry.x], popup=f"{data['name']}").add_to(m)
                continue
            elif geometry.geom_type in ['Polygon', 'MultiPolygon']:
                # Plot polygons or multipolygons
                color = amenity_colors[data.get('amenity')]
                if geometry.geom_type == 'Polygon':
                    polygons = [geometry]
                else:
                    polygons = geometry.geoms

                for polygon in polygons:
                    coordinates = []
                    for point in polygon.exterior.coords:
                        coordinates.append([point[1], point[0]])
                    folium.Polygon(locations=coordinates, fill=True, color=color, fill_opacity=0.4).add_to(m)

    # Return the map
    return m


In [143]:
# Function to connect zones in a network
def create_zone_network(graph, max_distance):
    connect_graph = nx.Graph()
    network_id = 1
    for node_key, node_data in list(graph.nodes.items()):
        if 'geometry' in node_data and node_data['geometry'].geom_type in ['Polygon', 'MultiPolygon']:
            connect_nodes = []
            for other_node_key, other_node_data in list(graph.nodes.items()):
                if 'geometry' in other_node_data and other_node_data['geometry'].geom_type in ['Polygon', 'MultiPolygon']:
                    if node_key != other_node_key:
                        
                        # Distance between the two centroids in meters
                        c1 = (node_data['geometry'].centroid.y, node_data['geometry'].centroid.x)
                        c2 = (other_node_data['geometry'].centroid.y, other_node_data['geometry'].centroid.x)
                        distance = geodesic(c1, c2).meters

                        if distance < max_distance:

                            # Check if any other amenities intersect the line between centroids
                            line_between_centroids = LineString([node_data['geometry'].centroid, other_node_data['geometry'].centroid])
                            amenities_intersecting = any(graph.nodes[amenity_key]['geometry'].intersects(line_between_centroids) for amenity_key in graph.nodes if amenity_key != node_key and amenity_key != other_node_key)
                            if not amenities_intersecting:
                                connect_nodes.append((other_node_key, node_data['geometry'].centroid.distance(other_node_data['geometry'])))
                                connect_network_lines(node_data['geometry'], other_node_data['geometry'])
                                
            # Adding to new graph if the current node has not been added 
            if node_key not in connect_graph.nodes:
                node_attributes = graph.nodes[node_key]
                node_attributes["network_id"] = network_id
                connect_graph.add_node(node_key, **node_attributes)
                
            # Connect them to one network
            for other_node_key, other_node_data in connect_nodes:
                if other_node_key not in connect_graph.nodes:
                    node_attributes = graph.nodes[node_key]
                    node_attributes["network_id"] = network_id
                    connect_graph.add_node(node_key, **node_attributes)
                connect_graph.add_edge(node_key, other_node_key)  # Add edges between nearby nodes
                
        network_id += 1
                
    return connect_graph


#FUNCTION FOR VISUALIZATION
def connect_network_lines(p1, p2):
    centroid1 = p1.centroid
    centroid2 = p2.centroid

    # Create a LineString from the centroids
    line_between_polygons = LineString([centroid1, centroid2])
    network_lines.append(line_between_polygons)

In [134]:
import random

# This is to better visualize the networks
def plot_connected_zones_network_on_map(graph, initial_location=[0, 0], zoom_start=10):
    # Create a map centered at the initial location
    map_center = (14.599512, 120.984222) # TEMPORARY WILL ZOOM TO MANILA
    m = folium.Map(location=map_center, zoom_start=zoom_start, tiles='openstreetmap')
    
    #Colours for Visualization
    colors = [
    "Red", "Green", "Blue", "Yellow", "Orange", "Purple", "Cyan", "Magenta", "Maroon",
    "Olive", "Lime", "Teal", "Navy", "Aqua", "Fuchsia", "Coral", "Indigo", "Lavender",
    "Violet", "DarkRed", "DarkGreen", "DarkBlue", "DarkOrange", "DarkPurple", "DarkCyan",
    "DarkMagenta", "RoyalBlue" , "Lavender"
]

    # Iterate over the nodes in the network
    for node, data in graph.nodes(data=True):
        # Check if the node has a geometry attribute
        if 'geometry' in data:
            # Get the geometry of the node
            geometry = data['geometry']

            # Check the geometry type and plot accordingly
            if geometry.geom_type == 'Point':
                # Plot a marker for points    
                #folium.Marker(location=[geometry.y, geometry.x], popup=f"{data['name']}").add_to(m)
                continue
            elif geometry.geom_type in ['Polygon', 'MultiPolygon']:
                # Plot polygons or multipolygons
                color = random.choice(colors)
                if geometry.geom_type == 'Polygon':
                    polygons = [geometry]
                else:
                    polygons = geometry.geoms

                for polygon in polygons:
                    coordinates = []
                    for point in polygon.exterior.coords:
                        coordinates.append([point[1], point[0]])
                    folium.Polygon(locations=coordinates, fill=True, color=color, fill_opacity=0.4).add_to(m)

    # Return the map
    return m


In [110]:
# Uses the combined graph
# Formula: Population Density = Total Population / Total Area

def check_residential_population_density(graph, threshold):
    # Create an empty array to store points
    points = []
    
    pop_graph = nx.Graph()
    for index, row in manila_population_gdf.iterrows():
        # Append the point to the points array
        points.append((row['latitude'], row['longitude'], row['phl_general_2020']))
    
    
    nodes = []
    for node_key, node_data in list(graph.nodes.items()):
        # Check if its a polygon and is a residential area
        if 'geometry' in node_data and node_data['geometry'].geom_type in ['Polygon', 'MultiPolygon'] and node_data['amenity'] == "residential areas":
            total_pop = 0
            
            for point in points:
                if Point(point[1], point[0]).within(node_data['geometry']):
                    total_pop += point[2] # Add the density
                    points.remove(point)
            
            density = total_pop / node_data['geometry'].area
            
            if (density > 0):
                node_data["is_a_zone"] = True
            else:
                node_data["is_a_zone"] = False
                
            nodes.append((node_key, density))
            pop_graph.add_node(node_key, **node_data)
                
    return pop_graph

In [136]:
# This is to better visualize which residential areas can become zones
def plot_population_zones_map(graph, initial_location=[0, 0], zoom_start=10):
    # Create a map centered at the initial location
    map_center = (14.599512, 120.984222) # TEMPORARY WILL ZOOM TO MANILA
    m = folium.Map(location=map_center, zoom_start=zoom_start, tiles='openstreetmap')

    # Iterate over the nodes in the network
    for node, data in graph.nodes(data=True):
        # Check if the node has a geometry attribute
        if 'geometry' in data:
            # Get the geometry of the node
            geometry = data['geometry']

            # Check the geometry type and plot accordingly
            if geometry.geom_type == 'Point':
                # Plot a marker for points    
                #folium.Marker(location=[geometry.y, geometry.x], popup=f"{data['name']}").add_to(m)
                continue
            elif geometry.geom_type in ['Polygon', 'MultiPolygon']:
                if geometry.geom_type == 'Polygon':
                    polygons = [geometry]
                else:
                    polygons = geometry.geoms

                for polygon in polygons:
                    coordinates = []
                    for point in polygon.exterior.coords:
                        coordinates.append([point[1], point[0]])
                    
                    if (data['is_a_zone']):
                        folium.Polygon(locations=coordinates, fill=True, color="green", fill_opacity=0.4).add_to(m)
                    else:
                        folium.Polygon(locations=coordinates, fill=True, color="red", fill_opacity=0.4).add_to(m)

    # Return the map
    return m

In [22]:
# Extract the graph into a geojson for loading into QGIS

def graph_to_geojson(graph, filename):
    # Initialize an empty list to hold GeoJSON features
    features = []

    # Iterate over the nodes in the graph
    for node, data in graph.nodes(data=True):
        # Check if the node has a geometry attribute
        if 'geometry' in data:
            # Convert the geometry to a GeoJSON-compatible format
            geometry = shapely.geometry.shape(data['geometry'])
            # Create a copy of the properties to check for NaN values
            properties = data.copy()
            # Remove the geometry from the properties
            properties.pop('geometry', None)
            # Check for NaN values in the properties
            if all(not (isinstance(value, float) and np.isnan(value)) for value in properties.values()):
                # Create a GeoJSON feature for the node
                feature = geojson.Feature(geometry=geometry, properties=properties)
                # Add the feature to the list
                features.append(feature)

    # Create a GeoJSON FeatureCollection
    feature_collection = geojson.FeatureCollection(features)

    # Return the GeoJSON FeatureCollection
    return feature_collection

## Main

In [137]:
# Creating Initial network
manila_amenities_network = create_network(manila_amenities_polygon_gdf, manila_amenities_point_gdf)

# Make a before map
before_map = plot_network_on_map(manila_amenities_network, initial_location=[0, 0], zoom_start=100)
before_map.save('before_map.html') # Save the map to an HTML file

In [138]:
# # Merging polygons of same amenity
# #Adjusted max_distance to 100 for testing
# merged_graph = merge_amenities_by_polygon(manila_amenities_network, max_distance=100, max_perimeter=5000)
# after_map = plot_network_on_map(merged_graph, initial_location=[0, 0], zoom_start=100)
# after_map.save('after_map.html') # Save the map to an HTML file

In [139]:
# Connecting polygons of same amenity
connected_lines = []
combined_graph = manila_amenities_network.copy()
combine_contiguous_amenities_by_polygon(combined_graph, max_distance=120, max_perimeter=10000)
after_map = plot_network_on_map(combined_graph, initial_location=[0, 0], zoom_start=100)


# The lines to show the networks
for line in connected_lines:
    line_coords = [[coord[1], coord[0]] for coord in line.coords]
    folium.PolyLine(locations=line_coords, color='black').add_to(after_map)
after_map.save('after_map.html') # Save the map to an HTML file

In [140]:
# Checking the population density of residential areas
pop_graph = check_residential_population_density(graph=combined_graph, threshold=100)
pop_map = plot_population_zones_map(pop_graph, initial_location=[0, 0], zoom_start=100)
pop_map.save('pop_map.html') # Save the map to an HTML file

In [141]:
# Create a network of amenities
network_lines = [] # This is to visualize which polygons are connected
graph_networks_of_polygons = create_zone_network(graph=combined_graph, max_distance=150)
networks_map = plot_connected_zones_network_on_map(graph_networks_of_polygons, initial_location=[0, 0], zoom_start=100)

# The lines to show the networks
for line in network_lines:
    line_coords = [[coord[1], coord[0]] for coord in line.coords]
    folium.PolyLine(locations=line_coords, color='black').add_to(networks_map)
networks_map.save('networks_map.html') # Save the map to an HTML file


feature_collection = graph_to_geojson(manila_amenities_network, 'output.geojson')
with open('output.geojson', 'w', encoding='utf-8') as f:
    f.write(geojson.dumps(feature_collection, indent=2))